# Lab Assignment One: Exploring Table Data 
**Christopher Cook, Bonita Davis, Anekah Kelley, Davis Lynn**

## 1. Business Understanding 

## 2. Data Understanding

## 3. Data Visualization

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/chrisc/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/chrisc/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1046, in launch_instance
    app.start()
  File "/home/chrisc/.local/lib/python3.10/site-pac

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [3]:
df = pd.read_csv('ClassicHit.csv')

df.head()

ModuleNotFoundError: No module named 'numpy.rec'

In [4]:
print(df.dtypes)

NameError: name 'df' is not defined

In [ ]:
# Create the JointGrid object
g = sns.JointGrid(data=df, x="Popularity", y="Danceability", space=0)

# Plot the 2D histogram with KDE on the margins
g.plot_marginals(sns.histplot, kde=True, bins=10)

# Set the aspect ratio of the x-axis marginal plot
#g.ax_marg_x.set_aspect(1.0 / g.ax_marg_x.get_data_ratio(), adjustable='box')

# Add the 2D histogram to the central plot
ax = g.ax_joint
sns.histplot(x='Popularity', y='Valence', data=df, bins=10, ax=ax)

# Now add text annotations to the histogram cells
coords = ax.collections[0].get_coordinates()
half_width = (coords[0, 1, 0] - coords[0, 0, 0]) / 2
half_height = (coords[1, 0, 1] - coords[0, 0, 1]) / 2

for v, (xv, yv) in zip(ax.collections[0].get_array().ravel(), coords[:-1, :-1, :].reshape(-1, 2)):
    if not np.ma.is_masked(v):
        ax.text(xv + half_width, yv + half_height, f'{v:.0f}', ha='center', va='center', color='white')

# Display the plot
plt.show()

## 4. Dimensionality Reduction